In [1]:
from __future__ import division
from numpy import *
from matplotlib.pyplot import *
import scipy.linalg
import pandas as pd
import csv
import time
import gc

## Functions for data and Time Serie generation

In [16]:
def loadData(file='data.csv'):
    """
Function to load data from file, returning the normalized data, unnorrmalized targets, mean and standard deviation
   :return: data,val_data,data mean and data standar deviation
    """
    data = np.genfromtxt('Resources/' + file, delimiter=',')
    return data

In [24]:
def getTimeSerie(file='data.csv',start_date="2001-01-01",periods = 0,freq = 'D'):
    """
Function to load Time Serie
    :param file: data file in Resources folder
    :param start_date: define start date of the Time Serie, is a string in format 'YYYY-MM-DD' by defaukt '2001-01-01'
    :param freq: frequency of the Time serie, by default is daily, visit  http://pandas.pydata.org/pandas-docs/stable/timeseries.html
    :return: Time Serie
    """
    data = loadData(file)
    index_ = pd.date_range(start_date,periods=len(data),freq=freq)
    TS= pd.Series(data,index = index_)
    return TS

### Function to Define Validation Train and Test TS

In [111]:
def getValidationTrainAndTestTS(TS,start=0,end=6,lenght=6):
    """
    Function to retrieve Validation Training and Test TS
    :param TS: Time Serie
    :param start: index from TS to start Training TS, by default = 0
    :param end: index from TS to end Training TS also used as start for Test TS, by default = 6
    :param lenght: lenght of Test TS, by default = 6, if is equal to end then will return to end of TS
    :return: Training TS, Test TS
    """
    try:
        trainTS = TS[start:-end].copy()
        if(lenght<start):
            testTS = TS[-end:-(end-lenght)]
        else:
            testTS = TS[-end:]
        return trainTS,testTS
    except Exception as e:
        print(e.value)         

### Function to Define Test Train TS

In [115]:
def getTestTrainTS(TS,start=0,end=None):
    """
Retrieve Test Training TS, by default the original TS
    :param TS: Time Serie
    :param start: index from original TS to start Test Training TS
    :param end: index from original TS to end Test Training TS, by default = None
    :return: Test Training TS
    """
    if(end is None):
        return TS[start:].copy()
    else:
        return TS[start:end].copy()

### Function to initilalize Win matrix

In [132]:
def generateWinMatrix(inSize, resSize, scaleFactor=1):
    """
Function to generate W in matrix with a uniform distribution between [-1,1]
in other Function the uniform distribution can be moved to any range
    :param inSize: number of inputs without interceptor
    :param resSize: size of reservoir
    :param scaleFactor: Scale Factor for the matrix by default 1
    :return: W in Matrix
    """
    Win = (random.uniform(low=-0.5, high=0.5, size=(resSize, 1 + inSize))) * scaleFactor
    return Win

### Function to initialize W matrix

In [131]:
def getSpectralRadius(W):
    """
Function to get the spectral radius of W matrix
    :param W: W matrix
    :return: spectral radius
    """
    rhoW = max(abs(linalg.eig(W)[0]))
    return  rhoW

In [135]:
def generateW(resSize, SRS = 1):
    """
Function to generate W matrix with a uniform distribution between [-1,1] and apply Spectral radius Scale
    :param resSize: size of reservoir
    :param SRS: spectral radius scale factor by default 1
    :return: W
    """
    W = random.uniform(low=-0.5, high= 0.5, size = (resSize,resSize))
    rhoW = get_spectral_radius(W)
    W*= (SRS/ rhoW)
    return W

### Function to get X and Yt matrix

In [136]:
def generateX_YtMatrix(data,inSize, resSize,initLen):
    """
Function to get X and Yt Matrix to save activation states and prediction targets
    :type data: normalized data
    :param inSize: size of the input 
    :param resSize: size of reservoir
    :param initLen:  length of initial data
    :return: X, Yt
    """
    X = zeros((1+inSize+resSize,len(data)-initLen))
    Yt = data[None,initLen+1:trainLen+1]
    return X,Yt

### Function to generate Win,W, X and Yt Matrix

In [ ]:
def generateMatrix(data,resSize,initLen = 12,inSize=1, WinScaleFactor=1, SRS = 1):
    """
    Function to generate Win, W, X and Yt matrix
    :param data: values of training Time Serie
    :param resSize: size of reservoir (num of neurons)
    :param initLen: number of values to be used as lags for activations states X matrix, by default = 12
    :param WinScaleFactor: scale factor for Win matrix, by default = 1
    :param SRS: Spectral radius scale factor for W matrix, by default = 1
    :return: Win,W,X,Xt
    """
    Win = generateWinMatrix(inSize,resSize,scaleFactor)
    W = generateW(resSize,SRS)
    X,Yt = generateX_YtMatrix(data,inSize,initLen)
    return Win,W,X,Yt

In [31]:
TS = getTimeSerie(freq='M')

In [116]:
TestTrainTS = getTestTrainTS(TS)

In [108]:
ValTrainTS,ValTestTS = getValidationTrainAndTestTS(TS)